In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return the number of GPUs
print(torch.cuda.get_device_name(0))  # Should show the GPU model

True
1
NVIDIA RTX 5000 Ada Generation


In [2]:
import transformers
import torch
import os
import json
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import logging
import time
from datetime import timedelta, datetime
import pandas as pd
from dotenv import load_dotenv
import shutil 

import evaluate
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load environment variables
load_dotenv(dotenv_path="../../.env") # path is relative to this script, adjust as needed

True

In [3]:
# setting huggingface token
login(token=os.getenv("HF_API_KEY"))
print("Hugging Face token set successfully.")
os.environ["HF_HOME"] = "D:/huggingface_cache" 
os.environ["TRANSFORMERS_CACHE"] = "D:/huggingface_cache"
os.environ["HUGGINGFACE_HUB_CACHE"] = "D:/huggingface_cache"
logging.info(f"Setting up Hugging Face environment variables...")

logging.info(f"HF_HOME: {os.getenv('HF_HOME')}")
logging.info(f"TRANSFORMERS_CACHE: {os.getenv('TRANSFORMERS_CACHE')}")
logging.info(f"HUGGINGFACE_HUB_CACHE: {os.getenv('HUGGINGFACE_HUB_CACHE')}")

transformers.utils.hub.TRANSFORMERS_CACHE = "D:/huggingface_cache"


model_name = "meta-llama/Meta-Llama-3-8B"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token if not set

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="cuda",
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Hugging Face token set successfully.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [4]:
model_name = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="cuda")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# test chunk
chunk = '''
Chapter 1: Introduction
Chapter 2: Photosynthesis and Plant Growth

Photosynthesis is the process by which green plants convert sunlight into energy. This energy is essential for plant growth and food production. The process takes place in the chloroplasts of plant cells, where sunlight, carbon dioxide, and water are converted into glucose and oxygen.

Section 5.1: Economic Impacts of Climate Change

Recent studies have shown that rising global temperatures and extreme weather events can significantly affect agricultural yields, infrastructure stability, and overall economic productivity in both developed and developing nations.

Table of Contents

Preface ............................................. v  
Chapter 1: Introduction ............................. 1  
Chapter 2: Background ............................... 5  
Chapter 3: Methodology .............................. 12  
References ......................................... 45

References

1. Smith, J. (2021). Introduction to Biology. Academic Press.  
2. Jones, M. & Lee, K. (2020). Climate Science: A Global Perspective. Springer.  
3. WHO Report on Air Quality, 2019.

Preface

This book provides an overview of key topics in environmental science. The reader is encouraged to explore the chapters in sequence, but each chapter can also be read independently based on interest.

'''

In [6]:
def build_prompt(chunk, questions_num):
    return f"""
You are a question generation model. Only generate questions if the text segment below contains useful, topic-relevant content. Ignore and do not generate questions for sections like prefaces, indexes, tables of contents, references, or bibliographies.

If the chunk is not useful, return the string: "This chunk is not useful."

Otherwise, generate {questions_num} question-answer pairs based on the following text segment. 
Return the result in valid JSON format as a list of objects.

Text Segment:

{chunk}

Response Format:
[
    {{"question": "generated question", "answer": "generated answer"}},
]

Each answer should be at least 250 words long.

Do NOT include any explanation or preamble before or after the JSON output.
Return ONLY valid JSON output or the string: "This chunk is not useful."

Answer:
    """


In [7]:
#  prompt with filtering instruction
prompt = build_prompt(chunk, 2)

# Send to model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

try:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output_tokens = model.generate(**inputs, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)
    
    generated_tokens = output_tokens[0][len(inputs["input_ids"][0]):]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    print("\n--- Chunk Being Tested ---\n")
    print(chunk)

    print("\n--- Model Output ---\n")
    print(generated_text)

    if generated_text == "This chunk is not useful.":
        print("\n✅ The model skipped this chunk because it identified it as non-useful content (e.g., references, preface, etc.).")
    else:
        try:
            qa_pairs = json.loads(generated_text)
            if isinstance(qa_pairs, list):
                print(f"\n⚠️ The model generated {len(qa_pairs)} question-answer pairs. It may not have correctly skipped this chunk.")
            else:
                print("\n⚠️ JSON output is not a list. Model may not have followed instructions.")
        except Exception as e:
            print(f"\n⚠️ JSON parse error: {e}. Model likely did not follow prompt instructions.")

except Exception as e:
    print(f"\n❌ Model generation error: {e}")



--- Chunk Being Tested ---


Chapter 1: Introduction
Chapter 2: Photosynthesis and Plant Growth

Photosynthesis is the process by which green plants convert sunlight into energy. This energy is essential for plant growth and food production. The process takes place in the chloroplasts of plant cells, where sunlight, carbon dioxide, and water are converted into glucose and oxygen.

Section 5.1: Economic Impacts of Climate Change

Recent studies have shown that rising global temperatures and extreme weather events can significantly affect agricultural yields, infrastructure stability, and overall economic productivity in both developed and developing nations.

Table of Contents

Preface ............................................. v  
Chapter 1: Introduction ............................. 1  
Chapter 2: Background ............................... 5  
Chapter 3: Methodology .............................. 12  
References ......................................... 45

References

1. Smith, J